# Heart rate analysis

This tutorial shows how to extract heart rate estimates using photoplethysmography (PPG) data and accelerometer data. The pipeline consists of a stepwise approach to determine signal quality, assessing both PPG morphology and accounting for periodic artifacts using the accelerometer. Based on the signal quality, we extract high-quality segments and estimate the heart rate for every 2 s using the smoothed pseudo Wigner-Ville Distribution. 

## Load data

This pipeline requires accelerometer and PPG data to run. In this example we loaded data from a participant of the Personalized Parkinson Project. We load the corresponding dataframes using the `load_tsdf_dataframe function`. The channel `green` represents the values obtained with PPG using green light.

In [ ]:
from pathlib import Path
from paradigma.util import load_tsdf_dataframe

path_to_data = Path('../../tests/data')
path_to_prepared_data = path_to_data / '1.prepared_data'

ppg_prefix = 'PPG'
imu_prefix = 'IMU'

df_ppg, _, _ = load_tsdf_dataframe(path_to_prepared_data / ppg_prefix, prefix = ppg_prefix)
df_imu, _, _ = load_tsdf_dataframe(path_to_prepared_data / imu_prefix, prefix = imu_prefix)

display(df_ppg, df_imu)

## Step 1: Preprocess data

The first step after loading the data is preprocessing. This begins by isolating segments containing both PPG and IMU data, discarding portions where one modality (e.g., PPG) extends beyond the other, such as when the PPG recording is longer than the accelerometer data. After this step, the preprocess_ppg_data function resamples the PPG and accelerometer data to uniformly distributed timestamps, addressing the fixed but non-uniform sampling rates of the sensors. After this, a bandpass Butterworth filter (4th-order, bandpass frequencies: 0.4--3.5 Hz) is applied to the PPG signal, while a high-pass Butterworth filter (4th-order, cut-off frequency: 0.2 Hz) is applied to the accelerometer data.

Note: the printed shapes are (rows, columns) with each row corresponding to a single data point and each column representing a data column (e.g.time). The number of rows of the overlapping segments of PPG and accelerometer are not exactly the same due to sampling differences (other sensors and possibly other sampling frequencies). 

In [ ]:
from paradigma.config import PPGConfig, IMUConfig
from paradigma.preprocessing import preprocess_ppg_data

ppg_config = PPGConfig()
imu_config = IMUConfig()

df_ppg_proc, df_acc_proc = preprocess_ppg_data(df_ppg, df_imu, ppg_config, imu_config)

display(df_ppg_proc, df_acc_proc)

## Step 2: Extract signal quality features

The preprocessed data (PPG & accelerometer) is windowed into overlapping windows of length `ppg_config.window_length_s` with a window step of `ppg_config.window_step_length_s`. From the PPG windows 10 time- and frequency domain features are extracted to assess PPG morphology and from the accelerometer windows one relative power feature is calculated to assess periodic motion artifacts. 

In [ ]:
from paradigma.config import HeartRateConfig
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features

ppg_config = HeartRateConfig('ppg')
acc_config = HeartRateConfig('imu')

print("The default window length for the signal quality feature extraction is set to", ppg_config.window_length_s, "seconds.")
print("The default step size for the signal quality feature extraction is set to", ppg_config.window_step_length_s, "seconds.")

df_features = extract_signal_quality_features(ppg_config, df_ppg_proc, acc_config, df_acc_proc)

df_features


## Step 3: Signal quality classification

A trained logistic classifier is used to predict PPG signal quality and returns the `pred_sqa_proba`, which is the posterior probability of a PPG window to look like the typical PPG morphology (higher probability indicates toward the typical PPG morphology). The relative power feature from the accelerometer is compared to a threshold for periodic artifacts and therefore `pred_sqa_acc_label` is used to return a label indicating predicted periodic motion artifacts (label 0) or no periodic motion artifacts (label 1). 

In [ ]:
from paradigma.heart_rate.heart_rate_analysis import signal_quality_classification

path_to_classifier = Path('../../tests/data/0.Classification/ppg')
config = HeartRateConfig()

df_sqa = signal_quality_classification(df_features, config, path_to_classifier)

df_sqa

## Step 4: Heart rate estimation

For heart rate estimation, we extract segments of `config.tfd_length`. We calculate the smoothed-pseudo Wigner-Ville Distribution (SPWVD) to obtain the frequency content of the PPG signal over time. We extract for every timestamp in the SPWVD the frequency with the highest power. For every non-overlapping 2 s window we average the corresponding frequencies to obtain a heart rate per window. 

In [ ]:
from paradigma.heart_rate.heart_rate_analysis import estimate_heart_rate

print("The default minimal window length for the heart rate extraction is set to", config.tfd_length, "seconds.")

df_hr = estimate_heart_rate(df_sqa, df_ppg_proc, config)

df_hr

## Step 5: Heart rate aggregation

The final step is to aggregate all 2 s heart rate estimates. In the current example, the mode and 99th percentile are calculated. We hypothesize that the mode gives representation of the resting heart rate while the 99th percentile indicates the maximum heart rate. In Parkinson's disease, we expect that these two measures could reflect autonomic (dys)functioning. The `nr_hr_est` in the metadata indicates based on how many 2 s windows these aggregates are determined.

In [ ]:
import pprint
from paradigma.heart_rate.heart_rate_analysis import aggregate_heart_rate

hr_values = df_hr['heart_rate'].values
df_hr_agg = aggregate_heart_rate(hr_values, aggregates = ['mode', '99p'])

pprint.pprint(df_hr_agg)